In [1]:
import pandas as pd
import numpy as np

import torch
from torch import nn
import sklearn
from sklearn.preprocessing import StandardScaler

# To find out if a python package is installed or not
# If not, installed it
import pkg_resources
import subprocess
import sys

def isinstall(packages_list):
    for package in packages_list:
        try:
            dist = pkg_resources.get_distribution(package)
            print('{} ({}) is installed'.format(dist.key, dist.version))
        except pkg_resources.DistributionNotFound:
            print('{} is NOT installed'.format(package))
            print('Installing...')
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
            print('Done!')

# Check and install Captum package (for model interpretability ) if necesssary
isinstall(['captum'])
import captum
from captum.attr import (
    IntegratedGradients
)

captum is NOT installed
Installing...


Done!


# Config values

In [2]:
# Threshold to take number of genes (biomarkers) per subtype
THRESHOLD = 100 
# COHORT: TCGA_BRCA, TCGA_LUNG, TCGA_CRC
COHORT = 'TCGA_BRCA'

In [3]:
assert COHORT in ['TCGA_BRCA','TCGA_LUNG', 'TCGA_CRC']
LIST_OMICS = ['GE', 'CNA']
ROOT_DATA_FOLDER = '../input/deepssc-omics-pretrained-model-and-biomarkers/DeepSSC_data/'
DATA_FOLDER = {'train':ROOT_DATA_FOLDER + f'{COHORT}/data/train/train_val_split/',
               'val':ROOT_DATA_FOLDER + f'{COHORT}/data/train/train_val_split/',
              'test':ROOT_DATA_FOLDER + f'{COHORT}/data/test/'}
PRETRAINED_MODEL = ROOT_DATA_FOLDER + f'{COHORT}/model/model/model.pt'
# Using unlabeled data to train autoencoder (z-norm with both labeled and unlabeled data) or not
USED_UNLABELED_DATA = (COHORT == 'TCGA_BRCA') | (COHORT == 'TCGA_CRC') 

LIST_TYPE_DATA = ['train', 'val', 'test']
DTYPE_INTEGER = 'int64'
DTYPE_FLOAT = 'float32'

# To be used to double check the accuracy on the test set
# To make sure the correct data and model are entered for the reproducibility of the results
ACC = {}
ACC['TCGA_BRCA'] = 0.8711656332015991
ACC['TCGA_LUNG'] = 0.9603960514068604
ACC['TCGA_CRC'] = 0.8867924809455872

# Load data and model

In [4]:
dict_df_label = {}
dict_df_data = {}
X = {}
y= {}
# Read data as df and create numpy array data for labeled data
for type_data in LIST_TYPE_DATA:
    dict_df_label[type_data] = pd.read_csv(DATA_FOLDER[type_data] + f'df_label_{type_data}.csv', index_col='sampleID')
    
    dict_df_omics = {}
    dict_narray_omics = {}
    for omic in LIST_OMICS:
        dict_df_omics[omic] = pd.read_csv(DATA_FOLDER[type_data] + f'df_{type_data}_{omic}_labeled.csv', index_col='sampleID')
        dict_narray_omics[omic] = dict_df_omics[omic].to_numpy(dtype=DTYPE_FLOAT, copy=True)
    dict_df_data[type_data] = dict_df_omics
    X[type_data] = dict_narray_omics

LABEL_MAPPING_NAME = dict_df_label['train']['disease_subtypes'].astype('category').cat.categories # sorted by alphabetical order
# Convert categorical label to numerical label
for type_data in LIST_TYPE_DATA:
    dict_df_label[type_data].loc[:,'disease_subtypes'] = dict_df_label[type_data]['disease_subtypes'].astype('category').cat.codes
    y[type_data] = np.squeeze(dict_df_label[type_data]['disease_subtypes'].to_numpy(dtype=DTYPE_INTEGER, copy=True))

# Read data as df and create numpy array data for unlabeled data
if USED_UNLABELED_DATA:
    dict_df_omics = {}
    dict_narray_omics = {}
    
    for omic in LIST_OMICS:
        dict_df_omics[omic] = pd.read_csv(DATA_FOLDER['train'] + f'df_train_{omic}_unlabeled.csv', index_col='sampleID')
        dict_narray_omics[omic] = dict_df_omics[omic].to_numpy(dtype=DTYPE_FLOAT, copy=True)
    dict_df_data['unlabeled'] = dict_df_omics
    X['unlabeled'] = dict_narray_omics
    
# z-norm
scaler = {}
for omic in LIST_OMICS: 
    scaler[omic] = StandardScaler()
    if USED_UNLABELED_DATA:
        scaler[omic].fit(
            np.concatenate((X['train'][omic], X['unlabeled'][omic]),axis=0)
        )
    else:
        scaler[omic].fit(X['train'][omic])
    for type_data in LIST_TYPE_DATA:
        X[type_data][omic] = scaler[omic].transform(X[type_data][omic])

In [5]:
class GEautoencoder(nn.Module):
    def __init__(self, fan_in):
        super(GEautoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(fan_in, 4096),
            nn.BatchNorm1d(4096),
            nn.ELU(),
            nn.Linear(4096, 2048),
            nn.ELU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(2048, 4096),
            nn.ELU(),
            nn.Linear(4096, fan_in)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

class CNVautoencoder(nn.Module):
    def __init__(self, fan_in):
        super(CNVautoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(fan_in, 4096),
            nn.BatchNorm1d(4096),
            nn.ELU(),
            nn.Linear(4096, 1024),
            nn.ELU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(1024, 4096),
            nn.ELU(),
            nn.Linear(4096, fan_in)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

class Subtyping_model(nn.Module):
    def __init__(self, ge_encoder, cnv_encoder, subtypes):
        super(Subtyping_model, self).__init__()
        
        self.ge_repr = nn.Sequential(*list(ge_encoder.children())[1:])
        self.cnv_repr = nn.Sequential(*list(cnv_encoder.children())[1:])
        
        self.classifier = nn.Sequential(
            nn.Linear(2048+1024, 1024),
            nn.ELU(),
            nn.Linear(1024, subtypes)
        )
        
    def forward(self, x1, x2):
        ge_ft = self.ge_repr(x1)
        cnv_ft = self.cnv_repr(x2)

        return self.classifier(torch.hstack((ge_ft, cnv_ft)))

In [6]:
# Load pretrained model
number_of_subtypes = len(LABEL_MAPPING_NAME)
classifier = Subtyping_model(GEautoencoder(fan_in = X['train']['GE'].shape[1]).encoder,
                             CNVautoencoder(fan_in = X['train']['CNA'].shape[1]).encoder,
                             subtypes = number_of_subtypes)
classifier.load_state_dict(torch.load(PRETRAINED_MODEL))
classifier.cuda()

Subtyping_model(
  (ge_repr): Sequential(
    (0): Linear(in_features=20530, out_features=4096, bias=True)
    (1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): Linear(in_features=4096, out_features=2048, bias=True)
    (4): ELU(alpha=1.0)
  )
  (cnv_repr): Sequential(
    (0): Linear(in_features=24776, out_features=4096, bias=True)
    (1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): Linear(in_features=4096, out_features=1024, bias=True)
    (4): ELU(alpha=1.0)
  )
  (classifier): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=1024, out_features=5, bias=True)
  )
)

In [7]:
# Double check the accuracy on the test set 
# To make sure the correct data and model are entered for the reproducibility of the results
print(np.unique(LABEL_MAPPING_NAME[y['val']], return_counts=True))
type_data = 'test'
_input = (torch.from_numpy(X[type_data][omic]).cuda() for omic in LIST_OMICS)
_target = torch.from_numpy(y[type_data]).cuda()
classifier.eval()
with torch.no_grad():
    pred = classifier(*_input)
    pred = torch.argmax(pred,dim=1)
    sum_true_label = torch.sum(pred == _target)
    number_samples = len(_target)
    acc = sum_true_label/number_samples
    print(f'acc = {sum_true_label}/{number_samples} = {acc}')
    
assert (acc == ACC[COHORT]), 'Cannot reproduce experimental results, check your data and model'

(array(['Basal', 'Her2', 'LumA', 'LumB', 'Normal'], dtype=object), array([23,  9, 61, 32,  5]))
acc = 142/163 = 0.8711656332015991


# Discover biomarkers

In [8]:
ig = IntegratedGradients(classifier)

In [9]:
# Remove str after "|" in gene id and create data to interpret (merge train and val dataset) 
gene_name = []
X_interpret = {}
for omic in LIST_OMICS:
    gene_name.extend(dict_df_data['train'][omic].columns.str.split(r'\|').str[0])
    X_interpret[omic] = np.concatenate((X['train'][omic], X['val'][omic]))
y_interpret = np.concatenate((y['train'],y['val']))

# Calculate attribute scores by batch data to avoid running out of memory
# 200 is the maximum (approximate) number of samples that will not cause run out of memory
max_samples_per_batch = 200
# Get list end of index to split data into batches
number_of_samples = len(y_interpret)
list_end_index = [max_samples_per_batch*times 
                  for times in range(1,int(number_of_samples/max_samples_per_batch))
                 ] + [number_of_samples]

# Calculate attribute score:
attr = {}
for subtype_idx, subtype in enumerate(LABEL_MAPPING_NAME):
    start = 0
    print(f'Calculate attribution scores with subtype {subtype}:')
    for end in list_end_index:
        print(f'\t samples from iloc {start} to {end}')
        
        input_tensor = (torch.from_numpy(X_interpret['GE'][start:end]).cuda().requires_grad_(),\
                         torch.from_numpy(X_interpret['CNA'][start:end]).cuda().requires_grad_())
        
        attr_temp, delta_temp = ig.attribute(input_tensor,
                                             target= subtype_idx, return_convergence_delta=True)
        # concatenate genes attribute score for multi-omics data
        attr_temp = np.concatenate((attr_temp[0].detach().cpu().numpy(),attr_temp[1].detach().cpu().numpy()), axis=1)
        if start == 0:
            attr[subtype] =  attr_temp
        else:
            attr[subtype] = np.concatenate((attr[subtype],attr_temp),axis=0)
        start=end

Calculate attribution scores with subtype Basal:
	 samples from iloc 0 to 200
	 samples from iloc 200 to 400
	 samples from iloc 400 to 656
Calculate attribution scores with subtype Her2:
	 samples from iloc 0 to 200
	 samples from iloc 200 to 400
	 samples from iloc 400 to 656
Calculate attribution scores with subtype LumA:
	 samples from iloc 0 to 200
	 samples from iloc 200 to 400
	 samples from iloc 400 to 656
Calculate attribution scores with subtype LumB:
	 samples from iloc 0 to 200
	 samples from iloc 200 to 400
	 samples from iloc 400 to 656
Calculate attribution scores with subtype Normal:
	 samples from iloc 0 to 200
	 samples from iloc 200 to 400
	 samples from iloc 400 to 656


In [10]:
df_attr = {}
# Build dataframe and rename gene id to gene name
for subtype in LABEL_MAPPING_NAME:
    df_attr[subtype] = pd.DataFrame(attr[subtype], columns=gene_name)

# Take mean all column (gene name) that have same name
for subtype in LABEL_MAPPING_NAME:
    df_attr[subtype] = df_attr[subtype].groupby(by=df_attr[subtype].columns, axis=1).mean()

In [11]:
# y predict by classifier/pretrained model
y_predict = torch.argmax(classifier(torch.from_numpy(X_interpret['GE']).cuda(),
                                       torch.from_numpy(X_interpret['CNA']).cuda()),
                                   axis=1).detach().cpu().numpy()
rank_score_by_subtype = {}
rank_score_by_subtype_POS = {}
rank_score_by_subtype_NEG = {}

matrix_top_threshold = pd.DataFrame()
for idx, subtype in enumerate(LABEL_MAPPING_NAME):
    temp_score = df_attr[subtype].loc[np.where((y_interpret == idx) &\
                                                        (y_predict == y_interpret)\
                                              )[0],:].mean(axis=0)
    rank_score_by_subtype_POS[subtype] = temp_score[temp_score>0].sort_values(ascending=False)
    temp_score = df_attr[subtype].loc[np.where((y_interpret != idx) &\
                                                        (y_predict == y_interpret)\
                                              )[0],:].mean(axis=0)
    
    rank_score_by_subtype_NEG[subtype] = temp_score[temp_score<0].sort_values(ascending=True).abs()
    rank_score_by_subtype[subtype] = rank_score_by_subtype_POS[subtype].append(
        rank_score_by_subtype_NEG[subtype])
    rank_score_by_subtype[subtype] = rank_score_by_subtype[subtype].sort_values(ascending=False)
    
    if len(LABEL_MAPPING_NAME) == 2:
        # Binary Classifier
        threshold_inner = THRESHOLD
        count_inner = 0
        while count_inner != THRESHOLD:
            count_inner = rank_score_by_subtype[subtype].iloc[:threshold_inner].index.nunique()
            if count_inner != THRESHOLD:
                threshold_inner = threshold_inner+1
            else:
                temp = pd.Series(list(rank_score_by_subtype[subtype].iloc[:threshold_inner].index))
                temp = temp[~temp.duplicated(keep='first')].reset_index(drop=True)
                matrix_top_threshold[subtype] = temp
    else:
        # Multi-class Classifier
        matrix_top_threshold[subtype] = list(rank_score_by_subtype[subtype].iloc[:THRESHOLD].index)
matrix_top_threshold.to_csv(f'matrix_biomarkers.csv',index=False)

In [12]:
matrix_top_threshold

,Basal,Her2,LumA,LumB,Normal
0,MGC3771,LOC150622,C10orf26,WDR67,OR5E1P
1,C6orf97,GRB7,WDR67,PPIL5,C1orf97
2,UPF0639,C17orf37,C9orf100,C12orf48,LOC728264
3,DLX6AS,ERBB2,FLJ36777,TIMELESS,OR2L13
4,C10orf26,STARD3,LASS4,CCDC99,CSN1S2A
...,...,...,...,...,...
95,FAM47E,KTELC1,FAM83D,RASSF10,LOC338758
96,FBN3,EPB49,METT5D1,GINS1,SLC27A6
97,PPP1CB,C17orf39,MEIS3P1,CASC5,IL22RA2
98,CDH19,COX7B2,C20orf114,FBXO5,NR3C1


In [13]:
print('Number of unique genes top 10:',len(set(matrix_top_threshold.iloc[:10,:].to_numpy(copy=True).reshape(-1).tolist())))
print('Number of unique genes top 100:',len(set(matrix_top_threshold.iloc[:100,:].to_numpy(copy=True).reshape(-1).tolist())))

Number of unique genes top 10: 46
Number of unique genes top 100: 428


In [14]:
print(np.__version__)
print(pd.__version__)
print(sklearn.__version__)
print(torch.__version__)
print(captum.__version__)

1.19.5
1.2.5
0.23.2
1.7.1+cu110
0.5.0
